# Explore product descriptions

Author: Felicia Liu

Date: August 12, 2019

In [10]:
import json
import pprint
import random

In [2]:
with open('product_data.json', 'r') as file:
    data = json.load(file)

In [5]:
product_descriptions = []
for index, product in enumerate(data):
    if 'description' in product:
        product_descriptions.append(product['description'])
    else:
        print("Missing description for product {}".format(index))

In [6]:
product_descriptions

["Supersoft speckled French terry makes this (tush covering!) turtleneck-sweatshirt hybrid the layering piece you'll want to wear to the gym, to lunch, to, well, everywhere this winter. Loose fit. Body length: 27 1/2. Cotton. Import.",
 'Consider this embroidered layer your new dressy sweatshirt this winter. Slim fit. Body length: 24. Cotton. Import.',
 "Shop the Men's Slim American Pima Cotton Oxford Shirt With Mechanical Stretch at J.Crew and see the entire selection of Men's Casual Shirts. Free Shipping Available.",
 "Shop the Women's Off-The-Shoulder Foldover Top at J.Crew and see the entire selection of Women's Knits. Free Shipping Available.",
 "Shop the Women's Ruffle Plunge V-Neck One-Piece Swimsuit at J.Crew and see the entire selection of Women's Swimwear. Free Shipping Available.",
 "Shop the Women's Strappy Knit Midi Dress at J.Crew and see the entire selection of Women's Swimwear. Free Shipping Available.",
 "Shop the Women's Mixed Stripe T-Shirt at J.Crew and see the enti

In [9]:
sample = random.sample(product_descriptions, 20)

Inspecting a random selection of product descriptions. Some descriptions are quite clean, but there are a number of descriptions that are either empty, not in English, or contain messy html.

In [13]:
for text in sample:
    print('*****')
    pprint.pprint(text)

*****
("Shop the Women's Tall Point Sur High-Rise Slim Boyfriend Jean In Destructed "
 "White Denim at J.Crew and see the entire selection of Women's Jeans. Free "
 'Shipping Available.')
*****
('Shop the Petite Boyfriend Shirt In Crimson Petal Plaid at JCrew.com and see '
 "our entire selection of Women's Shirts.")
*****
("Shop the Women's Signet Hobo Bag In Italian Leather at J.Crew and see the "
 "entire selection of Women's Bags. Free Shipping Available.")
*****
("Shop the Boys' New York Mets Jersey at J.Crew and see the entire selection "
 "of Boys' Tops. Free Shipping Available.")
*****
("Shop the Women's Surf Hipster Bikini Botton In Happy Seersucker at J.Crew "
 "and see the entire selection of Women's Swimwear. Free Shipping Available.")
*****
("Shop the Girls' Everyday Leggings In Metallic at J.Crew and see the entire "
 "selection of Girls' Pants. Free Shipping Available.")
*****
("Shop the Women's Petite Long Tie-Waist Jacket at J.Crew and see the entire "
 "selection of Wo

# Cleaning